In [1]:
import torch

In [2]:
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import random_split, TensorDataset 
import numpy as np
import pandas as pd
from sklearn.preprocessing import StandardScaler 


In [3]:
df = pd.read_excel("dataset_2.xlsx")
df.head()

,Nombre,Edad,Sexo,Altura,Peso,Entrena,Cantidad de estimulos,Plan
0,Paciente 1,65,0,154,67,1,3,1
1,Paciente 2,20,1,194,88,1,4,1
2,Paciente 3,27,0,195,99,1,1,0
3,Paciente 4,40,1,181,66,0,0,0
4,Paciente 5,49,1,192,99,1,2,1


In [4]:
y = df[df.columns[-1]]
y.head()

0    1
1    1
2    0
3    0
4    1
Name: Plan, dtype: int64

In [6]:
x = df.drop(columns=["Plan"])

In [7]:
x = x[x.columns[1:]]
x.head()

,Edad,Sexo,Altura,Peso,Entrena,Cantidad de estimulos
0,65,0,154,67,1,3
1,20,1,194,88,1,4
2,27,0,195,99,1,1
3,40,1,181,66,0,0
4,49,1,192,99,1,2


In [8]:
escalador = StandardScaler()
x= escalador.fit_transform(x)
x

array([[ 1.35404343, -1.1751393 , -1.58405136, -0.64095053,  0.85096294,
         1.19252255],
       [-1.39435325,  0.85096294,  1.9458681 ,  0.98073153,  0.85096294,
         1.90660791],
       [-0.96682488, -1.1751393 ,  2.03411609,  1.83018404,  0.85096294,
        -0.23564817],
       [-0.17284361,  0.85096294,  0.79864428, -0.71817348, -1.1751393 ,
        -0.94973353],
       [ 0.37683572,  0.85096294,  1.76937213,  1.83018404,  0.85096294,
         0.47843719],
       [ 1.35404343, -1.1751393 ,  0.0044124 ,  0.74906267, -1.1751393 ,
        -0.94973353],
       [-1.76080614,  0.85096294, -1.14281142, -1.33595713,  0.85096294,
         0.47843719],
       [ 0.1325338 , -1.1751393 ,  1.50462817,  0.67183971, -1.1751393 ,
        -0.94973353],
       [-1.82188162, -1.1751393 , -0.08383559,  1.67573813,  0.85096294,
         1.90660791],
       [ 0.01038283,  0.85096294,  0.53390032, -0.64095053,  0.85096294,
         0.47843719],
       [-1.2111268 ,  0.85096294, -0.78981948, -0.

In [9]:
x.shape[0]

100

In [10]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size =0.2,random_state = 2)

In [11]:
x_train.shape

(80, 6)

In [12]:
n_entradas = x.shape[1]

In [13]:
t_x_train = torch.from_numpy(x_train).float().to("cpu")
t_x_test = torch.from_numpy(x_test).float().to("cpu")
t_y_train = torch.from_numpy(y_train.values).float().to("cpu")
t_y_test = torch.from_numpy(y_test.values).float().to("cpu")
t_y_train = t_y_train[:,None] #
t_y_test = t_y_test[:,None]

In [14]:
t_y_train

tensor([[1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [0.],
        [1.],
        [1.],
        [1.],
        [1.],
        [0.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [0.],
        [1.],
        [1.],
        [0.],
        [1.],
        [0.],
        [1.],
        [1.],
        [0.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [0.],
        [1.],
        [1.],
        [0.],
        [1.],
        [1.],
        [1.],
        [0.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [1.],
        [0.],
        [0.],
        [1.],
        [1.],
        [1.],
        [1.],
        [0.],
        [1.],
      

In [15]:
class Red(nn.Module):
    def __init__(self, n_entradas):
        super(Red, self).__init__()
        self.linear1 = nn.Linear(n_entradas, 15)
        self.linear2 = nn.Linear(15, 8)
        self.linear3 = nn.Linear(8, 1)
    
    def forward(self, inputs):
        pred_1 = torch.sigmoid(input=self.linear1(inputs))
        pred_2 = torch.sigmoid(input=self.linear2(pred_1))
        pred_f = torch.sigmoid(input=self.linear3(pred_2))
        return pred_f
    

In [16]:
t_y_train[0]

tensor([1.])

In [17]:
lr = 0.001
epochs = 2000
estatus_print = 100

model = Red(n_entradas=n_entradas)
loss_fn = nn.BCELoss()
optimizer = torch.optim.Adam(params=model.parameters(), lr=lr)

print("Entranando el modelo")
for epoch in range(1, epochs+1):
    y_pred= model(t_x_train)
    loss = loss_fn(input=y_pred, target=t_y_train)
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()
    
    if epoch % estatus_print == 0:
        print(f"\nEpoch {epoch} \t Loss: {round(loss.item(), 4)}")
    
    with torch.no_grad():
        y_pred = model(t_x_test)
        y_pred_class = y_pred.round()
        correct = (y_pred_class == t_y_test).sum()
        accuracy = 100 * correct / float(len(t_y_test))
        if epoch % estatus_print == 0:
            print("Accuracy: {}".format(accuracy.item()))
    

Entranando el modelo

Epoch 100 	 Loss: 0.468
Accuracy: 55.0

Epoch 200 	 Loss: 0.4565
Accuracy: 55.0

Epoch 300 	 Loss: 0.4504
Accuracy: 55.0

Epoch 400 	 Loss: 0.4435
Accuracy: 55.0

Epoch 500 	 Loss: 0.4354
Accuracy: 55.0

Epoch 600 	 Loss: 0.4252
Accuracy: 55.0

Epoch 700 	 Loss: 0.4116
Accuracy: 55.0

Epoch 800 	 Loss: 0.3961
Accuracy: 55.0

Epoch 900 	 Loss: 0.3792
Accuracy: 50.0

Epoch 1000 	 Loss: 0.3602
Accuracy: 50.0

Epoch 1100 	 Loss: 0.3397
Accuracy: 50.0

Epoch 1200 	 Loss: 0.3179
Accuracy: 50.0

Epoch 1300 	 Loss: 0.2943
Accuracy: 50.0

Epoch 1400 	 Loss: 0.2704
Accuracy: 50.0

Epoch 1500 	 Loss: 0.2496
Accuracy: 50.0

Epoch 1600 	 Loss: 0.2328
Accuracy: 50.0

Epoch 1700 	 Loss: 0.2194
Accuracy: 50.0

Epoch 1800 	 Loss: 0.2073
Accuracy: 45.0

Epoch 1900 	 Loss: 0.1928
Accuracy: 45.0

Epoch 2000 	 Loss: 0.1702
Accuracy: 45.0
